<a href="https://colab.research.google.com/github/vodindo/SQL-Mini-Project-/blob/main/USA_Presidential_Elections.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
# Loading the SQL extension to the environment 
%load_ext sql

# We connect our memory sqlite database 

%sql sqlite://


'Connected: @None'

In [2]:

# Importing pandas 
import pandas as pd

In [3]:
# We load the Grand electors by state dataset

with open ('GrandElectors_by_state.csv', 'r') as f:
  GrandElectors = pd.read_csv(f, index_col=0, encoding='utf-8')

In [4]:
# Removing any similar table existing in our database

# Then we preview 
%sql DROP TABLE IF EXISTS GrandElectors;
%sql PERSIST GrandElectors;
%sql SELECT * FROM GrandElectors limit 5

 * sqlite://
Done.
 * sqlite://
 * sqlite://
Done.


State,GrandElectors
Alabama,9
Alaska,3
Arizona,11
Arkansas,6
California,55


In [5]:
# We equally Load the population by state dataset
with open ('Population_by_state.csv', 'r') as f:
  Population = pd.read_csv(f, index_col=0, encoding='utf-8')

In [6]:
# We then remove any similar tables and preview 


%sql DROP TABLE IF EXISTS Population;
%sql PERSIST Population;
%sql SELECT * FROM Population limit 5

 * sqlite://
Done.
 * sqlite://
 * sqlite://
Done.


State,Population
CALIFORNIA,39144818
TEXAS,27469114
FLORIDA,20271272
NEW YORK,19795791
ILLINOIS,12859995


In [7]:
# We convert ALL the States from GenderElectors to UPPER case
 
%%sql
UPDATE GrandElectors
SET State = UPPER(State)


 * sqlite://
51 rows affected.


[]

In [8]:
# Preview to check on the changes
%%sql
SELECT * from GrandElectors limit 5

 * sqlite://
Done.


State,GrandElectors
ALABAMA,9
ALASKA,3
ARIZONA,11
ARKANSAS,6
CALIFORNIA,55


In [9]:
# Join the two tables on the State key
# We do this by creating another table called JOINTTABLE 
%%sql
CREATE TABLE JOINTTABLE AS SELECT State, Population, GrandElectors 
FROM (SELECT * FROM Population
INNER JOIN GrandElectors ON 
Population.State = GrandElectors.State)

 * sqlite://
Done.


[]

In [10]:
%%sql
SELECT * FROM JOINTTABLE limit 5

 * sqlite://
Done.


State,Population,GrandElectors
CALIFORNIA,39144818,55
TEXAS,27469114,38
FLORIDA,20271272,29
NEW YORK,19795791,29
ILLINOIS,12859995,20


In [11]:
# We change the name "District of Columbia" state to "DC"
%%sql
UPDATE JOINTTABLE 
SET State = "DC"
WHERE State = "DISTRICT OF COLUMBIA"

 * sqlite://
1 rows affected.


[]

In [12]:

# create a new column to display that ratio between the number of grand electors and the population
%%sql
ALTER TABLE JOINTTABLE ADD RatioGP float(20)

 * sqlite://
Done.


[]

In [13]:
# Preview the table with added column
%%sql
SELECT * FROM JOINTTABLE limit 5

 * sqlite://
Done.


State,Population,GrandElectors,RatioGP
CALIFORNIA,39144818,55,None
TEXAS,27469114,38,None
FLORIDA,20271272,29,None
NEW YORK,19795791,29,None
ILLINOIS,12859995,20,None


In [14]:
# We Update the RatioGP column with the calculated ratio which is Population/GrandElectors
# this is then ordered by the states in decreasing ratio 

%%sql
UPDATE JOINTTABLE
SET "RatioGP" = Population/GrandElectors;
SELECT * FROM JOINTTABLE 
ORDER BY RatioGP ASC limit 5;

 * sqlite://
51 rows affected.
Done.


State,Population,GrandElectors,RatioGP
WYOMING,586107,3,195369.0
VERMONT,626042,3,208680.0
DC,672228,3,224076.0
ALASKA,738432,3,246144.0
NORTH DAKOTA,756927,3,252309.0


In [15]:
# Running total of Grand Electors in the sorted list.

%%sql
SELECT SUM(GrandElectors) from JOINTTABLE

 * sqlite://
Done.


SUM(GrandElectors)
538


In [16]:
# computing half of the total of Grand Electors overall (in the whole country):

%%sql
SELECT SUM(GrandElectors)/2 from JOINTTABLE

 * sqlite://
Done.


SUM(GrandElectors)/2
269


In [17]:
%%sql
SELECT State,GrandElectors,Population,
ROUND(GrandElectors*100.0/Population,6) AS Ratio
FROM JOINTTABLE
ORDER BY RATIO DESC;

 * sqlite://
Done.


State,GrandElectors,Population,Ratio
WYOMING,3,586107,0.000512
VERMONT,3,626042,0.000479
DC,3,672228,0.000446
ALASKA,3,738432,0.000406
NORTH DAKOTA,3,756927,0.000396
RHODE ISLAND,4,1056298,0.000379
SOUTH DAKOTA,3,858469,0.000349
DELAWARE,3,945934,0.000317
NEW HAMPSHIRE,4,1330608,0.000301
MAINE,4,1329328,0.000301


In [18]:
%%sql
SELECT T1.State, T1.GrandElectors, SUM(T2.GrandElectors) RunningTotal
FROM JOINTTABLE T1, JOINTTABLE T2
WHERE T1.GrandElectors <= T2.GrandElectors 
GROUP BY T1.State, T1.GrandElectors
ORDER BY T1.GrandElectors DESC, T1.State DESC;

 * sqlite://
Done.


State,GrandElectors,RunningTotal
CALIFORNIA,55,55
TEXAS,38,93
NEW YORK,29,151
FLORIDA,29,151
PENNSYLVANIA,20,191
ILLINOIS,20,191
OHIO,18,209
MICHIGAN,16,241
GEORGIA,16,241
NORTH CAROLINA,15,256
